In [21]:
import pyaudio
import wave
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib notebook
plt.rcParams['animation.html'] = 'jshtml'
import time
from pdb import set_trace
from scipy import signal
import librosa as lbr
import librosa.display

In [35]:
FORMAT = pyaudio.paInt16
CHANNELS = 1
Fs = 8000
CHUNK = 1024
REC_TIME = 5 #sec
audio = pyaudio.PyAudio()
# start Recording
stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=Fs, input=True,
                frames_per_buffer=CHUNK)
i=0
data = stream.read(CHUNK)  
data_int = decoded = np.frombuffer(data, np.int16);
voice=data_int
idx=0
acc=0
while True:
    idx+=1
    
    start=time.time()
    data = stream.read(CHUNK)  
    acc+=time.time()-start
    decoded = np.frombuffer(data, np.int16);
    voice=np.concatenate((voice,decoded))
    i+=1
    if(i == int(REC_TIME*Fs/CHUNK)):
        break
        
voice = voice[int(0.5*Fs):-1]
print('avg read time/chunk: ',acc/idx)
N=voice.shape[0]

avg read time/chunk:  0.12721946300604406


In [36]:
from scipy.ndimage import grey_erosion

In [40]:
S_WIND = int(0.01 * Fs)
N_FFT=2048
K=1
%matplotlib tk

#STFT
f,t,stft=signal.stft(voice,fs=Fs,nperseg=S_WIND,noverlap=0,return_onesided=True)
#Power spectrum probability
S=np.abs(stft)**2 #Energy
# S[np.logical_and(f<300,f>3000)] = 0

P=np.divide(S,np.sum(S,axis=0))
#Power spectrym entropy
H=-np.multiply(P,np.log2(P)).sum(axis=0)/np.log2(P.shape[0])

#Help distinguish with energy
E = S.sum(axis=0)
H = np.multiply(H,1-E/E.max())
H_size = H.shape[0]

plt.plot(np.array(range(N))/Fs,voice/voice.max())

plt.plot(t,H)
Hf = H
# Hf = signal.order_filter(Hf,domain=np.ones(9),rank=0)
Hf = signal.medfilt(H,kernel_size=7)
Hf = grey_erosion(Hf,size=(6))
plt.plot(t,Hf)
